# 📊 Session 3: Data Visualization Fundamental

**Durasi:** 90 menit  
**Dataset:** RUP Paket Penyedia 2025

## 🎯 Tujuan Pembelajaran
Setelah sesi ini, Anda dapat:
1. Membuat chart dasar dengan Plotly
2. Membuat bar chart, line chart, dan pie chart
3. Customize warna dan label
4. Save chart sebagai HTML/PNG

## 1️⃣ Setup

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path

# Load data
data_path = Path('../../../../datasets/rup/RUP-PaketPenyedia-Terumumkan-2025.parquet')
df = pd.read_parquet(data_path)

print(f"✅ Data loaded: {len(df):,} rows")

## 2️⃣ Bar Chart

In [ ]:
# Hitung jumlah paket per metode
metode_count = df['metode_pengadaan'].value_counts().reset_index()
metode_count.columns = ['metode', 'jumlah']

# Bar chart
fig = px.bar(
    metode_count,
    x='metode',
    y='jumlah',
    title='Jumlah Paket per Metode Pengadaan',
    labels={'metode': 'Metode Pengadaan', 'jumlah': 'Jumlah Paket'},
    color='jumlah',
    color_continuous_scale='Blues'
)

fig.update_layout(height=500)
fig.show()

In [ ]:
# Horizontal bar chart - Top 10 Satker
top_satker = df.groupby('nama_satker')['pagu'].sum().sort_values(ascending=False).head(10)
top_satker_df = pd.DataFrame({
    'satker': top_satker.index,
    'total_pagu_miliar': top_satker.values / 1e9
})

fig = px.bar(
    top_satker_df,
    y='satker',
    x='total_pagu_miliar',
    orientation='h',
    title='Top 10 Satker berdasarkan Total Pagu',
    labels={'satker': 'Satuan Kerja', 'total_pagu_miliar': 'Total Pagu (Miliar Rp)'},
    color='total_pagu_miliar',
    color_continuous_scale='Viridis'
)

fig.update_layout(height=500)
fig.show()

## 3️⃣ Pie Chart

In [ ]:
# Pie chart - Distribusi Jenis Pengadaan
jenis_count = df['jenis_pengadaan'].value_counts().head(5)

fig = px.pie(
    values=jenis_count.values,
    names=jenis_count.index,
    title='Distribusi Top 5 Jenis Pengadaan',
    hole=0.3  # Donut chart
)

fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(height=500)
fig.show()

## 4️⃣ Line Chart

In [ ]:
# Line chart - Tren pengumuman paket per bulan
df['tgl_pengumuman_paket'] = pd.to_datetime(df['tgl_pengumuman_paket'])
df['bulan'] = df['tgl_pengumuman_paket'].dt.to_period('M').astype(str)

monthly_count = df.groupby('bulan').size().reset_index(name='jumlah')

fig = px.line(
    monthly_count,
    x='bulan',
    y='jumlah',
    title='Tren Pengumuman Paket per Bulan',
    labels={'bulan': 'Bulan', 'jumlah': 'Jumlah Paket'},
    markers=True
)

fig.update_layout(height=400)
fig.show()

## 5️⃣ Histogram

In [ ]:
# Histogram - Distribusi Pagu
# Konversi ke juta untuk lebih mudah dibaca
df_plot = df[df['pagu'] > 0].copy()
df_plot['pagu_juta'] = df_plot['pagu'] / 1e6

fig = px.histogram(
    df_plot,
    x='pagu_juta',
    nbins=50,
    title='Distribusi Pagu Pengadaan',
    labels={'pagu_juta': 'Pagu (Juta Rp)', 'count': 'Frekuensi'},
    log_y=True  # Log scale pada Y-axis untuk melihat distribusi lebih jelas
)

fig.update_layout(height=400)
fig.show()

## 6️⃣ Grouped Bar Chart

In [ ]:
# Grouped bar - Metode vs Jenis (top combinations)
metode_jenis = df.groupby(['metode_pengadaan', 'jenis_pengadaan']).size().reset_index(name='jumlah')
metode_jenis = metode_jenis.nlargest(20, 'jumlah')

fig = px.bar(
    metode_jenis,
    x='metode_pengadaan',
    y='jumlah',
    color='jenis_pengadaan',
    title='Distribusi Jenis Pengadaan per Metode (Top 20)',
    labels={'metode_pengadaan': 'Metode', 'jumlah': 'Jumlah Paket'},
    barmode='group'
)

fig.update_layout(height=500)
fig.show()

## 7️⃣ Box Plot

In [ ]:
# Box plot - Distribusi pagu per metode
# Filter hanya metode dengan paket cukup banyak
top_metode = df['metode_pengadaan'].value_counts().head(5).index
df_filtered = df[df['metode_pengadaan'].isin(top_metode)]

fig = px.box(
    df_filtered,
    x='metode_pengadaan',
    y='pagu',
    title='Distribusi Pagu per Metode Pengadaan',
    labels={'metode_pengadaan': 'Metode', 'pagu': 'Pagu (Rp)'},
    log_y=True,
    color='metode_pengadaan'
)

fig.update_layout(height=500, showlegend=False)
fig.show()

## 8️⃣ Export Chart

In [ ]:
# Save sebagai HTML (interaktif)
fig.write_html('chart_metode_pagu.html')
print("✅ Chart saved as chart_metode_pagu.html")

# Save sebagai PNG (static) - requires kaleido package
# fig.write_image('chart_metode_pagu.png')

## 📊 Dashboard Mini: Subplots

In [ ]:
from plotly.subplots import make_subplots

# Prepare data
metode_count = df['metode_pengadaan'].value_counts().head(5)
jenis_count = df['jenis_pengadaan'].value_counts().head(5)

# Create subplots
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Distribusi Metode', 'Distribusi Jenis'),
    specs=[[{'type': 'pie'}, {'type': 'pie'}]]
)

# Add pie charts
fig.add_trace(
    go.Pie(labels=metode_count.index, values=metode_count.values, name='Metode'),
    row=1, col=1
)

fig.add_trace(
    go.Pie(labels=jenis_count.index, values=jenis_count.values, name='Jenis'),
    row=1, col=2
)

fig.update_layout(title_text='Dashboard Mini: Distribusi Pengadaan', height=400)
fig.show()

## 🎯 Latihan Mandiri

1. Buat bar chart untuk top 10 paket berdasarkan pagu
2. Buat line chart untuk tren pagu per bulan (bukan jumlah paket)
3. Buat pie chart untuk distribusi status PDN (PDN vs Non-PDN)
4. Buat stacked bar chart untuk metode pengadaan dengan breakdown per jenis

In [ ]:
# Ruang untuk latihan